# Map of the most GOAT Berlin neighborhoods.

GOAT score is calculated as the number of 🐐 homes per 1 km2 of each neighborhood. Fake goats were excluded from the analysis.


---


Goat homes were loacted by [Goats of Berlin](https://www.goatsofberlin.com)

Shapes of Berlin neighborhoods borrowed from [Michael Hörz](https://github.com/m-hoerz/berlin-shapes/blob/master/berliner-bezirke.geojson)

Map created with [Plotly](https://plotly.com/)

In [ ]:
%pip install area shapely plotly geopandas

In [ ]:
from shapely.geometry import Point


# coordinates of Berlin goat homes (https://goatsofberlin.com/)

goats = {
    "Children's Farm on the Görlitz e.V.": [52.49584950661747, 13.438292946678873],
    "Tierpark Neukölln in der Hasenheide": [52.486883598654, 13.417445329486883],
    "Tiergehege": [52.48915297815924, 13.380065706463258],
    "Kinderbauernhof Am Mauerplatz": [52.504951242097256, 13.421079804350144],
    "Blockinitiative 128": [52.51463217426161, 13.291184194555944],
    "Ziegen- und Hasenhof": [52.54271282408225, 13.282647227637433],
    "Kinderbunter Bauernhof": [52.54447179405055, 13.356909794555943],
    "Jugendfarm Moritzhof": [52.54842683054272, 13.401063082912298],
    "Ziegengehege": [52.568830179443125, 13.394087442979973],
    "Kinderbauernhof Knirpsenfarm": [52.57180142698459, 13.485370731336324],
    "Tierhof Marzahn": [52.54356047191263, 13.563660350377765],
    "Helle Tierarche e.V.": [52.54133447819932, 13.614680800104292],
    "Playground Koepenick": [52.46142790374832, 13.576155631336325],
    "WASLALA Kinderbauernhof": [52.40477224536035, 13.532859573665053],
    "Schloss Britz": [52.446079968390116, 13.438845689007598],
    "Domäne Dahlem": [52.458256100141995, 13.28969411599379],
    "Düppel": [52.4249744216459, 13.236753056473068],
    "Britzer Garten": [52.43249076133962, 13.412219973665062],
    "Vierfelderhof": [52.48257071430559, 13.170293086688902],
    "Reiterhof Kosa": [52.62028096856067, 13.420119462021415],
    "Gemeindepark Lankwitz": [52.43138193000489, 13.351704312294888],
    "Damwildgehege": [52.55206485357043, 13.325232754623618],
    "Haus Natur und Umwelt": [52.46434363213862, 13.543570496405392],
    "Tierpark Berlin": [52.50230095743021, 13.532218954076663],
    "Berlin Zoological Garden": [52.50806979261515, 13.337813708595997],
}

# these are goat statues
fake_goats = {
    "Goat statue at Zickenplatz": [52.49141035174273, 13.422298633185775],
    "Goat statue at Savignyplatz": [52.506240886112515, 13.32249475092472],
    "Goat statue by police office": [52.48570363525173, 13.30532883558218],
}

# convert to Points
goat_points = [Point(coord[1], coord[0]) for coord in goats.values()]
fake_goat_points = [Point(coord[1], coord[0]) for coord in fake_goats.values()]

In [ ]:
import geopandas as gpd
from area import area

# Germany Zone 4 https://epsg.io/?q=Germany
utm_crs = 'ESRI:31494'  # UTM CRS for the chosen zone
berlin_data = gpd.read_file('https://raw.githubusercontent.com/m-hoerz/berlin-shapes/master/berliner-bezirke.geojson')

# index by bezirk (neighborhood) name
berlin_data['bezirk'] = berlin_data['spatial_alias']
berlin_data.set_index("bezirk", inplace=True)

# https://gis.stackexchange.com/questions/218450/getting-polygon-areas-using-geopandas
berlin_data['area'] = berlin_data['geometry'].to_crs(utm_crs).area/ 10**6

# get number of goat homes
berlin_data['goat_points'] = 0

# Iterate through each polygon
for index, polygon in berlin_data.iterrows():
    # Iterate through each point
    for point in goat_points:
        # Check if the point is inside the polygon
        if point.within(polygon['geometry']):
            berlin_data.at[index, 'goat_points'] += 1

# GOAT score calculated as number of goats per km2
berlin_data['GOAT score'] = (berlin_data['goat_points']/berlin_data['area']).round(3)

# berlin_data.sort_values(by='GOAT score', ascending=False)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# add bezirks to the map (color based on goat density)
fig = px.choropleth_mapbox(berlin_data,
                           geojson=berlin_data.geometry,
                           locations=berlin_data.index,
                           color='GOAT score',
                           opacity= 0.7,
                           color_continuous_scale='Brwnyl',
                           center={"lon": 13.42, "lat": 52.50},
                           mapbox_style="open-street-map",
                           zoom=9.7)

# add goats
goat_trace = go.Scattermapbox(
    mode = "markers",
    lon = [str(round(g[1], 5)) for g in goats.values()],
    lat = [str(round(g[0], 5)) for g in goats.values()],
    marker = go.scattermapbox.Marker(
            size=10,
            color='black'
        ),
    text = 'Meeh' # because it is what goats say
    )
fig.add_trace(goat_trace)


fig.update_layout(
    width=1000,
    height=1000,
    font_family="Courier New",
    font_color="black",
    title=go.layout.Title(
        text="Berlin's best neighborhoods based on the GOAT score (goatsofberlin.com) <br><sup>GOAT score calculated as the number of 🐐 homes per km2</sup>",
        xref="paper",
        x=0
    ),
)

fig.show()